<a href="https://colab.research.google.com/github/LxMera/CNN-3D-data/blob/main/get_data_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
#function ClickConnect(){ console.log("Working2"); document.querySelector("colab-toolbar-button#connect").click() }setInterval(ClickConnect,10000)

In [34]:
#!mkdir /content/drive/My\ Drive/rsfMRI

In [ ]:
'''Dirs=['https://www.fmrib.ox.ac.uk/datasets/FIX-training/HCP_hp2000_runs1-20.tar.gz',
'https://www.fmrib.ox.ac.uk/datasets/FIX-training/HCP_hp2000_runs21-40.tar.gz',
'https://www.fmrib.ox.ac.uk/datasets/FIX-training/HCP_hp2000_runs41-60.tar.gz',
'https://www.fmrib.ox.ac.uk/datasets/FIX-training/HCP_hp2000_runs61-80.tar.gz',
'https://www.fmrib.ox.ac.uk/datasets/FIX-training/HCP_hp2000_runs81-100.tar.gz',
'https://www.fmrib.ox.ac.uk/datasets/FIX-training/HCP_hp2000_run1.tar.gz',
'https://www.fmrib.ox.ac.uk/datasets/FIX-training/WhII_MB6.tar.gz',
'https://www.fmrib.ox.ac.uk/datasets/FIX-training/WhII_Standard.tar.gz',
'https://www.fmrib.ox.ac.uk/datasets/FIX-training/Standard.tar.gz']'''

In [36]:
'''import os
for path in Dirs:
  print('Download '+path[path.rfind('/')+1:])
  os.system('wget '+path+' -P /content/drive/My\ Drive/rsfMRI/')
  os.system('tar -xzvf /content/drive/My\ Drive/rsfMRI'+path[path.rfind('/'):]+' -C /content/drive/My\ Drive/rsfMRI/')
  os.system('rm -r /content/drive/My\ Drive/rsfMRI'+path[path.rfind('/'):])'''

"import os\nfor path in Dirs:\n  print('Download '+path[path.rfind('/')+1:])\n  os.system('wget '+path+' -P /content/drive/My\\ Drive/rsfMRI/')\n  os.system('tar -xzvf /content/drive/My\\ Drive/rsfMRI'+path[path.rfind('/'):]+' -C /content/drive/My\\ Drive/rsfMRI/')\n  os.system('rm -r /content/drive/My\\ Drive/rsfMRI'+path[path.rfind('/'):])"

In [ ]:
!pip install  nilearn

In [38]:
DirP='/content/drive/My Drive/rsfMRI'
Folders=['/HCP_hp2000', '/Standard', '/WhII_MB6', '/WhII_Standard']

In [39]:
import glob
import numpy as np
import matplotlib.pyplot as plt
from nilearn import image
import cv2

conver=np.vectorize(int)

In [40]:
suball=[]
for fold in enumerate(Folders):
  Subjets=glob.glob(DirP+fold[1]+'/*')
  if fold[0]==0:
    suball=Subjets
  else:
    suball=np.concatenate((suball,Subjets))

  print(np.shape(Subjets)[0],' subjets in the folder '+ fold[1])
print(np.shape(suball))

100  subjets in the folder /HCP_hp2000
106  subjets in the folder /Standard
39  subjets in the folder /WhII_MB6
45  subjets in the folder /WhII_Standard
(290,)


In [41]:
def get_time_series(path, matrix=False):
  series=np.loadtxt(path+'/filtered_func_data.ica/melodic_mix')
  time_points, ICs=np.shape(series)
  if matrix:
    return ICs,  time_points, series
  else:
    return ICs

In [42]:
def get_maps_ica(path):
  ima_gz=image.image.load_img(path+'/filtered_func_data.ica/melodic_IC.nii.gz')
  maps_ica=ima_gz.get_fdata()
  return(maps_ica)

In [43]:
def vector_hand_labels_noise(path, n_comp):
  label=np.ones(n_comp)
  f = open(sub+"/hand_labels_noise.txt")
  while True:
    line = f.readline()
    if line=='':
      break
    handClas=line[1:-2]
  handClas= handClas.split(", ")
  handClas=conver(handClas)-1
  label[handClas]=0
  f.close()
  return label

In [44]:
def contour_like_vector(im_var):
  im_bin=np.array((im_var!=im_var[0,0]), np.dtype('uint8'))
  contours,_ = cv2.findContours(im_bin,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
  ind=0
  if np.shape(contours)[0]!=1:
    areas=[]
    for xi in range(np.shape(contours)[0]):
      area=cv2.contourArea(contours[xi])
      areas.append(area)
    ind=np.argmax(areas)
  cont=contours[ind]
  rectangle=cv2.boundingRect(cont)
  return rectangle

In [45]:
#this function is to determine volume axes and to verified if don't exist mistakes
def get_axis(minx):
  Paxi=minx[0]
  Pcor=minx[1]
  Psag=minx[2]
  sg=Ds=cr=Dc=ax=Dx=0

  if Paxi[1]==Pcor[1] and Paxi[3]==Pcor[3]:
    sg=Paxi[1]
    Ds=Paxi[3]
  else:
    sg=np.min((Paxi[1],Pcor[1]))
    Ds=np.max((Paxi[3],Pcor[3]))

  if Paxi[0]==Psag[1] and Paxi[2]==Psag[3]:
    cr=Paxi[0]
    Dc=Paxi[2]
  else:
    cr=np.min((Paxi[0],Psag[1]))
    Dc=np.max((Paxi[2],Psag[3]))

  if Psag[0]==Pcor[0] and Psag[2]==Pcor[2]:
    ax=Pcor[0]
    Dx=Pcor[2]
  else:
    ax=np.min((Psag[0],Pcor[0]))
    Dx=np.max((Psag[2],Pcor[2]))

  return sg, sg+Ds, cr,cr+Dc, ax, ax+Dx

In [46]:
def brain_region(mps, plot_slice=False):
  sx, cy, az, n_comp = np.shape(mps)
  varx=np.zeros((cy,az))
  vary=np.zeros((sx,az))
  varz=np.zeros((sx,cy))

  for ncp in range(n_comp):
    varx=varx+np.var(mps[:,:,:,ncp], axis=0)
    vary=vary+np.var(mps[:,:,:,ncp], axis=1)
    varz=varz+np.var(mps[:,:,:,ncp], axis=2)

  conx=contour_like_vector(varx)
  cony=contour_like_vector(vary)
  conz=contour_like_vector(varz)

  roi_3d=get_axis((conz,cony,conx))

  if plot_slice:
    plt.imshow(varx)
    plt.figure()
    plt.imshow(vary)
    plt.figure()
    plt.imshow(varz)

  return roi_3d

In [47]:
from scipy.ndimage import zoom
def adjust_volume(mps, SAG=57, COR=73, AXI=57):
  roi=brain_region(mps)
  mps=mps[roi[0]:roi[1],roi[2]:roi[3],roi[4]:roi[5],:]
  x0,y0,z0,_=np.shape(mps)
  mp = zoom(mps, (SAG/x0, COR/y0, AXI/z0, 1))
  mp = np.transpose(mp)
  return mp

In [48]:
sub=suball[0]
ics,t_p, series=get_time_series(sub, True)
noise=vector_hand_labels_noise(sub, ics)
maps=get_maps_ica(sub)
maps=adjust_volume(maps)

print('time points: ', t_p)
print('Number of independent componets: ', ics)
print('Shape of map: ',  np.shape(maps))

time points:  1200
Number of independent componets:  250
Shape of map:  (250, 57, 73, 57)


In [49]:
import scipy.io as sio
outpath='/content/drive/My Drive/rsfMRI/Comprimidos/'

In [ ]:
for sub in suball[206:]:
  print(sub[31:].replace('/','-').replace('ica','mat'))

  ics,t_p, series=get_time_series(sub, True)
  Y_train=vector_hand_labels_noise(sub, ics)
  maps=get_maps_ica(sub)
  X_train=adjust_volume(maps)
  
  sio.savemat(outpath+sub[31:].replace('/','-').replace('ica','mat'), {'maps': X_train, 'labels': Y_train, 'times': series})

WhII_MB6-2.mat
WhII_MB6-3.mat
WhII_MB6-4.mat
WhII_MB6-5.mat
WhII_MB6-6.mat
WhII_MB6-7.mat
